In [1]:
import numpy as np
# from scipy.stats.qmc import Halton
import matplotlib.pyplot as plt
import trimesh
import geopandas as gpd
import math
import pandas as pd

In [2]:
# calc interval each sequence and check if all interval have same value
def isIncreasing(seq):
   return all(a<b for a,b in zip(seq,seq[1:]))

seq = [1,2,3]
# [[a, b] for a,b in zip(seq,seq[1:])]
dinterval = [b-a for a,b in zip(seq,seq[1:])]

dinterval.count(dinterval[0]) == len(dinterval)

True

In [3]:
collar_map = {
    'x': 'UTM_EAST',
    'y': 'UTM_NORTH',
    'z': 'RL',
    'depth': 'TDEPTH',
}

collar_map.get('depth')

'TDEPTH'

In [4]:
collars = gpd.read_file('./BAKAN_COLLAR.CSV')
# collars
surveys = gpd.read_file('./BAKAN_SURVEY.CSV')
# surveys
assays = gpd.read_file('./BAKAN_ASSAY.CSV')
# assays

In [5]:
assays.columns

Index(['DATE_LAB', 'BATCH_ID', 'BHID', 'OLD_BHID', 'FROM', 'TO', 'SAMPLE',
       'AU_FINAL', 'AUCN_FINAL', 'AG_FINAL', 'AUCN/AUFA', 'AG/AU', 'AU1',
       'AU2', 'AU3', 'AU4', 'AU5', 'AUCN', 'AUCN1', 'AUCN2', 'AG1', 'AG2',
       'CU', 'PB', 'ZN', 'AS', 'SB', 'BI', 'FE', 'MO', 'HG', 'W', 'C_%', 'S_%',
       'UTM_EAST', 'UTM_NORTH', 'RL', 'AREA', 'geometry'],
      dtype='object')

In [6]:
agg = surveys.groupby(['AREA', 'PROJECT', 'BHID'])
agg.count()

AT  MINE_BRG  MAG_BRG  UTM_BRG  DM_DIP  MM_DIP  \
AREA       PROJECT   BHID                                                      
           BAKAN     MRP1033   1         1        1        1       1       1   
                     MRP1034   1         1        1        1       1       1   
                     MRP1035   1         1        1        1       1       1   
                     MRP1036   1         1        1        1       1       1   
                     MRP1037   1         1        1        1       1       1   
...                           ..       ...      ...      ...     ...     ...   
WEST VILLA BKN_DRILL WVD900    9         9        9        9       9       9   
                     WVLD410   5         5        5        5       5       5   
                     WVLD411   6         6        6        6       6       6   
                     WVLD412   5         5        5        5       5       5   
                     WVLD413   5         5        5        5       5       5   

                              geometry  
AREA       PROJECT   BHID               
           BAKAN     MRP1033         0  
                     MRP1034         0  
                     MRP1035         0  
                     MRP1036         0  
                     MRP1037         0  
...                                ...  
WEST VILLA BKN_DRILL WVD900          0  
                     WVLD410         0  
                     WVLD411         0  
                     WVLD412         0  
                     WVLD413         0  

[1118 rows x 7 columns]

In [7]:
agg_keys = list(agg.groups.keys())

# agg_keys[0]

# iterate over agg_keys to create segmented line with tvd
for agg_key in agg_keys:
    print(agg_key)

('', 'BAKAN', 'MRP1033')
('', 'BAKAN', 'MRP1034')
('', 'BAKAN', 'MRP1035')
('', 'BAKAN', 'MRP1036')
('', 'BAKAN', 'MRP1037')
('', 'BAKAN', 'MRP1038')
('', 'BAKAN', 'MRP1039')
('', 'BAKAN', 'MRP1040')
('', 'BAKAN', 'MRP1041')
('', 'BAKAN', 'MRP1042')
('', 'BAKAN', 'MRP1043')
('', 'BAKAN', 'MRP1044')
('', 'BAKAN', 'MRP1045')
('', 'BAKAN', 'MRP1046')
('', 'BAKAN', 'MRP1047')
('', 'BAKAN', 'MRP1048')
('', 'BAKAN', 'MRP1049')
('', 'BAKAN', 'MRP1050')
('', 'BAKAN', 'MRP1051')
('', 'BAKAN', 'MRP1052')
('', 'BAKAN', 'MRP1053')
('', 'BAKAN', 'MRP1054')
('', 'BAKAN', 'MRP1055')
('', 'BAKAN', 'MRP1056')
('', 'BAKAN', 'MRP1057')
('', 'BAKAN', 'MRP1058')
('', 'BAKAN', 'MRP1059')
('', 'BAKAN', 'MRP1060')
('', 'BAKAN', 'MRP1061')
('', 'BAKAN', 'MRP1062')
('', 'BAKAN', 'MRP1063')
('', 'BAKAN', 'MRP1064')
('', 'BAKAN', 'MRP1065')
('', 'BAKAN', 'MRP1066')
('', 'BAKAN', 'MRP1066R')
('', 'BAKAN', 'MRP1067')
('', 'BAKAN', 'MRP1068')
('', 'BAKAN', 'MRP1069')
('', 'BAKAN', 'MRP1070')
('', 'BAKAN', 'MRP1070R'

In [8]:
collar = collars.query('BHID=="WVD900"')
# collar

In [9]:
# survey = agg.get_group(('WEST VILLA', 'BKN_DRILL', 'WVD900'))
survey = surveys.query('BHID=="WVD900"')
# survey

In [10]:
assay = assays.query('BHID=="WVD900"')
# assay

In [11]:
survey['AT'] = survey.apply(lambda row: float(row['AT']), axis=1)

assay['FROM'] = assay.apply(lambda row: float(row['FROM']), axis=1)
assay['TO'] = assay.apply(lambda row: float(row['TO']), axis=1)

collar[collar_map.get('x')] = collar.apply(
    lambda row: float(row[collar_map.get('x')]), axis=1
)
collar[collar_map.get('y')] = collar.apply(
    lambda row: float(row[collar_map.get('y')]), axis=1
)
collar[collar_map.get('z')] = collar.apply(
    lambda row: float(row[collar_map.get('z')]), axis=1
)
collar[collar_map.get('depth')] = collar.apply(
    lambda row: float(row[collar_map.get('depth')]), axis=1
)

/media/lelegoyang/OS/DevTeam/jupyter_env/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [12]:
BHID = 'WVD900'
collar_info = {
    'id': BHID,
    'x': collar[collar_map.get('x')].iloc[0:1].values[0],
    'y': collar[collar_map.get('y')].iloc[0:1].values[0],
    'z': collar[collar_map.get('z')].iloc[0:1].values[0],
}
survey_info = {
    'id': BHID,
}
survey_map = {
    'depth': 'AT',
    'dip': 'DM_DIP',
    'azimuth': 'UTM_BRG',
}

interval_info = {
    'id': BHID,
    'z0': 'FROM', # top interval
    'z1': 'TO', # bottom interval
}

In [13]:
collar[collar_map.get('x')].iloc[0:1].values[0]

645379.481

# I WOULD LIKE TO PLOT INTERVALS LINE IN GLB

input dataframe of single borehole  
output the glb stuff yes

## SOME CONCERN
`collar_map` should have column information mapping  
`collar_info` should have the value (information)

In [14]:
# for each interval, calc TVD (true z) from intervals depth + survey
z0 = [float(item) for item in assay[interval_info['z0']]]
z1 = [float(item) for item in assay[interval_info['z1']]]

# get overlaping intervals
z1Overlap = [a < b for a,b in zip(z0[1:], z1)]
z1Overlap.append(False) # additional for last interval
assay['z1Overlap'] = np.asarray(z1Overlap)

# get gaping intervals
z0Gap = [a>b for a,b in zip(z0[1:], z1)]
z0Gap.insert(0, False)
assay['z0Gap'] = np.asarray(z0Gap)

# should be empty, skipped tho
# assay.query('z1Overlap==1')
# assay.query('z0Gap==1')

assay

,DATE_LAB,BATCH_ID,BHID,OLD_BHID,FROM,TO,SAMPLE,AU_FINAL,AUCN_FINAL,AG_FINAL,...,W,C_%,S_%,UTM_EAST,UTM_NORTH,RL,AREA,geometry,z1Overlap,z0Gap
57515,,BKNEXP-160,WVD900,,0.0,1.0,D864128BKN,0.003,,,...,,,,645379.233,63006.666,713.938,MAIN RIDGE,None,False,False
57516,,,WVD900,,1.0,2.0,D864129BKN,0.018,,,...,,,,645378.738,63006.668,713.069,MAIN RIDGE,None,False,False
57517,,,WVD900,,2.0,3.0,D864130BKN,0.012,,,...,,,,645378.242,63006.670,712.200,MAIN RIDGE,None,False,False
57518,,,WVD900,,3.0,4.3,D864131BKN,0.003,,,...,,,,645377.673,63006.672,711.201,MAIN RIDGE,None,False,False
57519,,,WVD900,,4.3,5.3,D864132BKN,0.003,,,...,,,,645377.103,63006.674,710.203,MAIN RIDGE,None,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57645,,,WVD900,,199.9,201.9,D864272BKN,0.003,,,...,,,,645281.720,63009.476,538.897,MAIN RIDGE,None,False,False
57646,,,WVD900,,201.9,203.9,D864273BKN,0.003,,,...,,,,645280.740,63009.538,537.155,MAIN RIDGE,None,False,False
57647,,,WVD900,,203.9,205.9,D864274BKN,0.003,,,...,,,,645279.760,63009.599,535.412,MAIN RIDGE,None,False,False
57648,,,WVD900,,205.9,207.9,D864275BKN,0.003,,,...,,,,645278.780,63009.661,533.670,MAIN RIDGE,None,False,False


In [15]:
# case DIP AT 0 is >>>>>>

# get dip value at depth, None if not found
def get_dip(survey_df, depth):
    survey_at = survey_df.query(
        '{z}<={value}'.format(
            z=survey_map.get('depth'),
            value=depth,
        )
    ).sort_values(
        survey_map.get('depth'),
        ascending=False
    ).iloc[0:1]
    # survey_at
    len(survey_at)
    dip = survey_at.get(survey_map['dip'])
    try:
        return float(dip)
    except:
        return None

print(get_dip(survey, 0))
print(get_dip(survey, 100))
print(get_dip(survey, -10))

60.0
61.1
None


In [16]:
# calc true vertical depth by length
# examples:
# 1meter 90deg dip has tvd 1meter
# 1meter 0deg dip has tvd 0meter

def tvd_length(length=0, dip=90):
    return length * np.sin(np.radians(dip))

def tvd_displacement(length=0, dip=90):
    return length * np.cos(np.radians(dip))

def get_tvd_len(row):
    length = row['TO'] - row['FROM']
    return tvd_length(length, row['dip'])

def get_tvd_displace(row):
    length = row['TO'] - row['FROM']
    return tvd_displacement(length, row['dip'])

assay['dip'] = assay.apply(lambda row: get_dip(survey, row['FROM']), axis=1)
assay['tvdLen'] = assay.apply(lambda row: get_tvd_len(row), axis=1)
assay['tvdDisplacement'] = assay.apply(lambda row: get_tvd_displace(row), axis=1)

# leLen = [b-a for a,b in zip(z0, z1)]
# assay

/media/lelegoyang/OS/DevTeam/jupyter_env/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


leFrom = assay['FROM'].values.tolist()
leTo = assay['TO'].values.tolist()
tvdLen = assay['tvdLen'].values.tolist()

### THE TVD OBJECT
```python
tvd = {
    xi: [x0, x1], # west-east displacement relative from collar point
    yi: [y0, y1], # north-south displacement relative from collar point
    zi: [z0, z1], # depth displacement (tvd) relative from collar point
}
```

In [17]:
# case azimuth at 0 is >>>>>>>
# get dip value at depth, None if not found
def get_azimuth(survey_df, depth):
    survey_at = survey_df.query(
        '{depth}<={value}'.format(
            value=depth,
            depth=survey_map.get('depth'),
        )
    ).sort_values(
        survey_map.get('depth'),
        ascending=False
    ).iloc[0:1]
    # survey_at
    len(survey_at)
    az = survey_at.get(survey_map['azimuth'])
    try:
        return float(az)
    except:
        return None

print(get_azimuth(survey, 0))
print(get_azimuth(survey, 100))
print(get_azimuth(survey, -10))

270.0
271.5
None


In [18]:
# calc displacement after tvd (idk what's the word but yes)
def x_displacement(displacement=0, azimuth=90):
    return displacement * np.sin(np.radians(azimuth))

def y_displacement(displacement=0, azimuth=90):
    return displacement * np.cos(np.radians(azimuth))

def get_x_displacement(row):
    return x_displacement(row['tvdDisplacement'], get_azimuth(survey, row['FROM']))

def get_y_displacement(row):
    return y_displacement(row['tvdDisplacement'], get_azimuth(survey, row['FROM']))

xdisp = assay.apply(lambda row: get_x_displacement(row), axis=1)
ydisp = assay.apply(lambda row: get_y_displacement(row), axis=1)
tvdLen = assay['tvdLen'].values

In [19]:
tvdXI = [[np.sum(xdisp[:i]), np.sum(xdisp[:i+1])] for i, item in enumerate(xdisp)]
tvdYI = [[np.sum(ydisp[:i]), np.sum(ydisp[:i+1])] for i, item in enumerate(ydisp)]
tvdZI = [[np.sum(tvdLen[:i]), np.sum(tvdLen[:i+1])] for i, item in enumerate(tvdLen)]
# tvdXI
# tvdYI
# tvdZI

In [20]:
true_displacement = [
    {
        'xi': a,
        'yi': b,
        'zi': c,
    }
    for a,b,c in zip(tvdXI, tvdYI, tvdZI)
]

assay['trueDisp'] = true_displacement

/media/lelegoyang/OS/DevTeam/jupyter_env/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [21]:
assay['trueDisp'].values.tolist()[0].get('xi')

[0.0, -0.5000000000000001]

In [22]:
def get_line_segment(row):
    x = collar_info.get('x')
    y = collar_info.get('y')
    z = collar_info.get('z')
    
    segmentX = list(map(lambda dx: x + dx, row['trueDisp'].get('xi')))
    segmentY = list(map(lambda dy: y + dy, row['trueDisp'].get('yi')))
    segmentZ = list(map(lambda dz: z + dz, row['trueDisp'].get('zi')))
    return [[a,b,c] for a,b,c in zip(segmentX, segmentY, segmentZ)]

assay['segment'] = assay.apply(lambda row: get_line_segment(row), axis=1)
assay

,DATE_LAB,BATCH_ID,BHID,OLD_BHID,FROM,TO,SAMPLE,AU_FINAL,AUCN_FINAL,AG_FINAL,...,RL,AREA,geometry,z1Overlap,z0Gap,dip,tvdLen,tvdDisplacement,trueDisp,segment
57515,,BKNEXP-160,WVD900,,0.0,1.0,D864128BKN,0.003,,,...,713.938,MAIN RIDGE,None,False,False,60.0,0.866025,0.500000,"{'xi': [0.0, -0.5000000000000001], 'yi': [0.0,...","[[645379.481, 63006.665, 714.372], [645378.981..."
57516,,,WVD900,,1.0,2.0,D864129BKN,0.018,,,...,713.069,MAIN RIDGE,None,False,False,60.0,0.866025,0.500000,"{'xi': [-0.5000000000000001, -1.00000000000000...","[[645378.981, 63006.665, 715.2380254037844], [..."
57517,,,WVD900,,2.0,3.0,D864130BKN,0.012,,,...,712.200,MAIN RIDGE,None,False,False,60.0,0.866025,0.500000,"{'xi': [-1.0000000000000002, -1.50000000000000...","[[645378.481, 63006.665, 716.1040508075688], [..."
57518,,,WVD900,,3.0,4.3,D864131BKN,0.003,,,...,711.201,MAIN RIDGE,None,False,False,60.0,1.125833,0.650000,"{'xi': [-1.5000000000000004, -2.15000000000000...","[[645377.981, 63006.665, 716.9700762113532], [..."
57519,,,WVD900,,4.3,5.3,D864132BKN,0.003,,,...,710.203,MAIN RIDGE,None,False,False,60.0,0.866025,0.500000,"{'xi': [-2.1500000000000004, -2.65000000000000...","[[645377.331, 63006.665, 718.0959092362731], [..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57645,,,WVD900,,199.9,201.9,D864272BKN,0.003,,,...,538.897,MAIN RIDGE,None,False,False,60.7,1.744139,0.978765,"{'xi': [-97.39894006545063, -98.37507647350807...","[[645282.0820599346, 63009.03345072058, 888.90..."
57646,,,WVD900,,201.9,203.9,D864273BKN,0.003,,,...,537.155,MAIN RIDGE,None,False,False,60.6,1.742428,0.981808,"{'xi': [-98.37507647350807, -99.35494660789544...","[[645281.1059235266, 63009.10513369752, 890.65..."
57647,,,WVD900,,203.9,205.9,D864274BKN,0.003,,,...,535.412,MAIN RIDGE,None,False,False,60.6,1.742428,0.981808,"{'xi': [-99.35494660789544, -100.3348167422827...","[[645280.1260533922, 63009.16678190098, 892.39..."
57648,,,WVD900,,205.9,207.9,D864275BKN,0.003,,,...,533.670,MAIN RIDGE,None,False,False,60.6,1.742428,0.981808,"{'xi': [-100.33481674228278, -101.314686876670...","[[645279.1461832577, 63009.228430104435, 894.1..."


In [65]:
p.polygons_closed

array([<shapely.geometry.polygon.Polygon object at 0x7f8cd2417b80>],
      dtype=object)

In [66]:
s = trimesh.Scene()

le_segs = assay['segment'].values.tolist()
circ = trimesh.path.creation.circle(radius=2)
rect = trimesh.path.creation.rectangle(bounds=[[0, 0], [2, 2]])
# p = trimesh.path.util.concatenate([rect])
p = trimesh.path.util.concatenate([circ])
polygon = p.polygons_closed[0]

# s.add_geometry(p)

yuh = trimesh.creation.sweep_polygon(
    polygon=polygon,
    path=le_path,
)

s.add_geometry(yuh)

s.show(viewer='notebook')

/tmp/ipykernel_211518/1743175245.py:12: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  yuh = trimesh.creation.sweep_polygon(


In [24]:
s = trimesh.Scene()

le_segs = assay['segment'].values.tolist()
p = trimesh.load_path(le_segs)
p.colors = [[0, 0, 0] for _ in p.entities]

s.add_geometry(p)

s.show(viewer='notebook')

In [25]:
s = trimesh.Scene()

cylinder = trimesh.creation.cylinder(radius=0.1, height=0.3)
cylinder.visual.face_colors = [0, 0, 1., 0.5]
s.add_geometry(cylinder)

s.show(viewer='notebook')

In [67]:
le_byte = trimesh.exchange.gltf.export_glb(s)
f = open("sweep.glb", "wb")
f.write(le_byte)
f.close()

In [26]:
segment = assay['segment']
assays = pd.concat([assays, segment], axis=1)

In [27]:
assays['segment'] = assays.apply(lambda row: None if np.isnan(row['segment']).all() else row['segment'], axis=1)

In [28]:
assay2 = assays.query('BHID=="WVD900"')
assay2

,DATE_LAB,BATCH_ID,BHID,OLD_BHID,FROM,TO,SAMPLE,AU_FINAL,AUCN_FINAL,AG_FINAL,...,HG,W,C_%,S_%,UTM_EAST,UTM_NORTH,RL,AREA,geometry,segment
57515,,BKNEXP-160,WVD900,,0.00,1.00,D864128BKN,0.003,,,...,,,,,645379.233,63006.666,713.938,MAIN RIDGE,None,"[[645379.481, 63006.665, 714.372], [645378.981..."
57516,,,WVD900,,1.00,2.00,D864129BKN,0.018,,,...,,,,,645378.738,63006.668,713.069,MAIN RIDGE,None,"[[645378.981, 63006.665, 715.2380254037844], [..."
57517,,,WVD900,,2.00,3.00,D864130BKN,0.012,,,...,,,,,645378.242,63006.670,712.200,MAIN RIDGE,None,"[[645378.481, 63006.665, 716.1040508075688], [..."
57518,,,WVD900,,3.00,4.30,D864131BKN,0.003,,,...,,,,,645377.673,63006.672,711.201,MAIN RIDGE,None,"[[645377.981, 63006.665, 716.9700762113532], [..."
57519,,,WVD900,,4.30,5.30,D864132BKN,0.003,,,...,,,,,645377.103,63006.674,710.203,MAIN RIDGE,None,"[[645377.331, 63006.665, 718.0959092362731], [..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57645,,,WVD900,,199.90,201.90,D864272BKN,0.003,,,...,,,,,645281.720,63009.476,538.897,MAIN RIDGE,None,"[[645282.0820599346, 63009.03345072058, 888.90..."
57646,,,WVD900,,201.90,203.90,D864273BKN,0.003,,,...,,,,,645280.740,63009.538,537.155,MAIN RIDGE,None,"[[645281.1059235266, 63009.10513369752, 890.65..."
57647,,,WVD900,,203.90,205.90,D864274BKN,0.003,,,...,,,,,645279.760,63009.599,535.412,MAIN RIDGE,None,"[[645280.1260533922, 63009.16678190098, 892.39..."
57648,,,WVD900,,205.90,207.90,D864275BKN,0.003,,,...,,,,,645278.780,63009.661,533.670,MAIN RIDGE,None,"[[645279.1461832577, 63009.228430104435, 894.1..."
